# Coursera Capstone Week 3 Assignment

### Segmenting and Clustering Neighborhoods in Toronto

In [3]:
!pip install geocoder

#Import necessary packages
import pandas as pd
import geocoder

     |████████████████████████████████| 102kB 4.1MB/s 


First, we need to scrape the table from Wikipedia. We will do this using pandas.

In [4]:
#use read_html to import tables on the page
dfs = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
print(dfs[0].head())

  Postal Code           Borough              Neighbourhood
0         M1A      Not assigned               Not assigned
1         M2A      Not assigned               Not assigned
2         M3A        North York                  Parkwoods
3         M4A        North York           Victoria Village
4         M5A  Downtown Toronto  Regent Park, Harbourfront


In [6]:
#check to see if there are more tables
print(dfs[1].head())

                                                  0   ...   17
0                                                NaN  ...  NaN
1  NL NS PE NB QC ON MB SK AB BC NU/NT YT A B C E...  ...  NaN
2                                                 NL  ...   YT
3                                                  A  ...    Y

[4 rows x 18 columns]


It looks like the first table is the one we want. In the next code cell, we'll select the first table (index 0)

### Requirements for data preparation:
- The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood
- Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.
- More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11  in the above table.
- If a cell has a borough but a Not assigned  neighborhood, then the neighborhood will be the same as the borough.
- Clean your Notebook and add Markdown cells to explain your work and any assumptions you are making.
- In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.

In [9]:
df = dfs[0]

In [13]:
# Filter out the cells that have 'Not assigned' in the Borough column
df = df[df.Borough != "Not assigned"]

# Check result
print(df.head(10))

   Postal Code           Borough                                Neighbourhood
2          M3A        North York                                    Parkwoods
3          M4A        North York                             Victoria Village
4          M5A  Downtown Toronto                    Regent Park, Harbourfront
5          M6A        North York             Lawrence Manor, Lawrence Heights
6          M7A  Downtown Toronto  Queen's Park, Ontario Provincial Government
8          M9A         Etobicoke      Islington Avenue, Humber Valley Village
9          M1B       Scarborough                               Malvern, Rouge
11         M3B        North York                                    Don Mills
12         M4B         East York              Parkview Hill, Woodbine Gardens
13         M5B  Downtown Toronto                     Garden District, Ryerson


Conveniently, the rows are already combined where Neighbourhoods share a single Postal Code. We don't need to do any additional processing for this. 

In [16]:
# Check for rows where Neighbourhood has a value of 'Not assigned'
print(df[df.Neighbourhood == "Not assigned"])

Empty DataFrame
Columns: [Postal Code, Borough, Neighbourhood]
Index: []


There are no entries in the 'Neighbourhood' column where the value is 'Not assigned'. We don't need to do any processing here, either.

### Conclusion
We now have a cleaned dataframe.
- [x] Three columns: Postal Code, Borough, and Neighborhood
- [x] Only process cells with an assigned borough
- [x] Where neighbourhoods share a postal code, include them in a single entry for the postal code in the 'Neighbourhood' column, separated by commas
- [x] If the 'Neighbourhood' value is 'Not assigned', change it to be the same as the value in the 'Borough' column

In [18]:
df.shape

(103, 3)